In [1]:
# data science
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer

import youtube.get_youtube_data as get_youtube_data
import machine_learning.embedding as embedding
import save_dislikes.sentiment as sentiment

In [2]:
# Step 1. Get the Youtube API key. The Youtube API key created in Google Cloud by following the instructions on
#         the API overview page: https://developers.google.com/youtube/v3/getting-started
YOUTUBE_API_KEY = get_youtube_data.get_youtube_api_key()

In [3]:
# Step 2. Build the Youtube client to make API calls.
youtube = get_youtube_data.make_client(YOUTUBE_API_KEY)

In [4]:
# Step 3. Perform a Youtube search with a user-specified query.
youtube_df = get_youtube_data.search_youtube(
    youtube,
    query='Police Brutality',
    max_vids=15,       # only grab at most 50 to reduce quota usage
    order='relevance', # default is relevance
    comments=False,
    max_comments=20,   # only grab at most 100 to reduce quota usage
    transcripts=False,
)

Searching for: Police Brutality
Returning 15 results


In [5]:
youtube_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   video_id                15 non-null     object 
 1   published_at            15 non-null     object 
 2   channel_id              15 non-null     object 
 3   title                   15 non-null     object 
 4   description             15 non-null     object 
 5   channel_title           15 non-null     object 
 6   live_broadcast_content  15 non-null     object 
 7   thumbnail_default_url   15 non-null     object 
 8   thumbnail_medium_url    15 non-null     object 
 9   thumbnail_high_url      15 non-null     object 
 10  thumbnail_standard_url  15 non-null     object 
 11  thumbnail_maxres_url    14 non-null     object 
 12  tags                    14 non-null     object 
 13  video_category_id       15 non-null     int64  
 14  video_duration          15 non-null     floa

In [6]:
youtube_df.sample(5)

,video_id,published_at,channel_id,title,description,channel_title,live_broadcast_content,thumbnail_default_url,thumbnail_medium_url,thumbnail_high_url,...,tags,video_category_id,video_duration,video_caption,video_view_count,video_like_count,video_comment_count,is_comments_enabled,is_live_content,NoCommentsBinary
7,9a7ADRgu7XE,2023-06-29T16:10:28Z,UCfmkz6A3aTJwJzox2xhAuRw,School district withholding video of alleged p...,"In Las Vegas, the ACLU is filing legal action ...",WRAL,none,https://i.ytimg.com/vi/9a7ADRgu7XE/default.jpg,https://i.ytimg.com/vi/9a7ADRgu7XE/mqdefault.jpg,https://i.ytimg.com/vi/9a7ADRgu7XE/hqdefault.jpg,...,"[wral, wral tv, las vegas, Clark County School...",25,149.0,False,11016.0,392.0,171.0,1,0,0
4,WFArdwbGZ2w,2023-07-20T17:13:33Z,UCHjm6wybRbldhqveS7c2WTA,​​Why NYPD Paid $13 Million To Make A Police B...,Black Lives Matter protesters won a record $13...,Insider News,none,https://i.ytimg.com/vi/WFArdwbGZ2w/default.jpg,https://i.ytimg.com/vi/WFArdwbGZ2w/mqdefault.jpg,https://i.ytimg.com/vi/WFArdwbGZ2w/hqdefault.jpg,...,"[Insider, News, Black Lives Matter, NYPD, Geor...",25,581.0,False,41936.0,532.0,398.0,1,0,0
9,3cgY4KKZ92s,2022-07-22T09:55:25Z,UCDmNmxF3ZVMeGyvWE9tOqPQ,Humble man claims police brutality during arre...,A man from Humble says he was the victim of po...,ABC13 Houston,none,https://i.ytimg.com/vi/3cgY4KKZ92s/default.jpg,https://i.ytimg.com/vi/3cgY4KKZ92s/mqdefault.jpg,https://i.ytimg.com/vi/3cgY4KKZ92s/hqdefault.jpg,...,[news],25,204.0,True,59441.0,1321.0,708.0,1,0,0
0,Vb-9e0uF_BA,2023-07-24T00:24:32Z,UCinjnmQEwCddOudyCC1v7qA,Transgender man violently arrested by LASD deputy,Surveillance footage from a 7-Eleven store in ...,KTLA 5,none,https://i.ytimg.com/vi/Vb-9e0uF_BA/default.jpg,https://i.ytimg.com/vi/Vb-9e0uF_BA/mqdefault.jpg,https://i.ytimg.com/vi/Vb-9e0uF_BA/hqdefault.jpg,...,"[video, news, 5live]",25,164.0,True,22708.0,390.0,896.0,1,0,0
3,nDPk_nK8454,2023-07-28T09:01:02Z,UCqBJ47FjJcl61fmSbcadAVg,"Uhuru Kenyatta, Azimio leaders attend requiem ...",Subscribe and watch NTV Kenya live for latest ...,NTV Kenya,none,https://i.ytimg.com/vi/nDPk_nK8454/default.jpg,https://i.ytimg.com/vi/nDPk_nK8454/mqdefault.jpg,https://i.ytimg.com/vi/nDPk_nK8454/hqdefault.jpg,...,"[kenya news, kenyan news today, kenyan latest ...",25,53.0,False,15435.0,295.0,21.0,1,0,0


In [11]:
import joblib
from time import perf_counter

print("Loading model...")
start_load_time = perf_counter()
# Path to load the model
model_pickle_path = 'save_dislikes/rfclf.joblib.pkl'
rf_clf = joblib.load(model_pickle_path)
print(f"  Time taken to load model: {(perf_counter() - start_load_time):.4f} seconds")
print()

youtube_df = sentiment.sort_by_sentiment(rf_clf, youtube_df)

Loading model...
  Time taken to load model: 2.5201 seconds

Making predictions...
  Time taken to make predictions: 0.0235 seconds


In [8]:
filter_sent = "Politics"
list_of_videos = ["Who'se Really Supporting Russia",
                  "The Perfect Hillary Clinton Analogy",
                  "The Evolution of Alex Jones",\
                  "Patrick Bet David on The Breakfast Club",
                  "The Truth About The 2020 Election",
                  "Kobe Bryant's Last Great Interview"]
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embedding.filter_out_embed(model,filter_sent,list_of_videos)


['Patrick Bet David on The Breakfast Club',
 "Kobe Bryant's Last Great Interview"]

In [9]:
titles = youtube_df['title'].tolist()
print("titles:",titles)
print("api test")
print(embedding.filter_out_embed(model,filter_sent,titles))

titles: ['Transgender man violently arrested by LASD deputy', 'Indianapolis Police Sergeant Facing Charges After Bodycam Shows Excessive Force', 'Chief refutes claims of police brutality in Southfield', 'Uhuru Kenyatta, Azimio leaders attend requiem for victims of police brutality', '\u200b\u200bWhy NYPD Paid $13 Million To Make A Police Brutality Lawsuit Go Away | Insider News', 'L.I. man suing Nassau County Police over alleged police brutality', 'Video shows alleged police brutality in Tamarac', 'School district withholding video of alleged police brutality at Las Vegas high school', 'Man held down by #police in #Walmart after being accused of stealing pizza', 'Humble man claims police brutality during arrest caught on video', 'Who is attracted to policing jobs? | Brian Klaas', 'Newly released police misconduct investigations show SDPD&#39;s struggle with racial discrimination', 'Police Officers Suspended After Violent Arrest Is Caught On Camera', 'How a Violent Arrest Has School Pol